# Graph convolutional networks for partial atomic charges

## Authors
* John D. Chodera `<john.chodera@choderalab.org>`


## Abstract

The assignment of consistent, reproducible, high-quality partial atomic charges for biomolecular systems is frustrated by the reliance of popular methods (such as AM1-BCC and RESP) on quantum chemical methods, which suffer from multiple challenges: (1) they require a reproducible way to generate molecular conformations in order to ensure reproducible charges; (2) large molecules such as biopolymers require fragmentation and capping in a manner that does not perturb their electronic structure; and (3) the expense of even semiempirical quantum chemical computations can become prohibitive for large biopolymers or small molecule datasets. Here, we present an fast alternative approach to rapidly deriving partial atomic charges (which could function as the initial stage for a bond charge correction scheme) that uses only the molecular topology, thereby avoiding the need to reproducibly generate molecular conformations. Instead of sloq quantum chemical calculations, a fast graph convolutional neural network is used to estimate parameters for a simple graph-based charge computation scheme that respects resonance structures and preserves total molecular charge. 

## Motivation

The assignment of partial atomic charges for biomolecular simulations has traditionally utilized one of two methods:
1. An _electrostatic potential fitting_ approach such as [RESP](http://doi.org/10.1021/j100142a004) that enumerates one or more molecular conformations, computes the electrostatic potential on multiple shells around the molecule, and performs a restraint fit to the electrostatic potential
2. A _bond charge correction_ scheme such as [AM1-BCC](goo.gl/2HPt1e) [(see also)](https://doi.org/10.1002/jcc.10128) that enumerates one or more molecular conformations, performs a semi-empirical quantum chemical calculation followed by population analysis to obtain inexpensive but crude partial charge estimates, and then corrects them with a set of rules that alter charges across bonds

These methods suffer from several drawbacks:
* They require enumeration of molecular conformations. In order to reproducibly assign charges, these conformations must also be reproducibly computed, which presents a highly challenging problem to ensure across molecular cheminformatics toolkits.
* They utilize quantum chemical calculations, which are inherently slow, even for semiempirical calculations.
* Large biopolymers, such as proteins and nucleic acids, are too slow to perform direct quantum chemical computations, requiring either fragmentation and capping in a manner that preserves electronic properties or the use of linear-scaling (such as divide-and-conquer) quantum chemical approaches. Neither problem is trivial to solve.

Here, we consider whether the molecular topology---that is, the molecular graph containing information about the identity and chemical connecivity of atoms in a small molecule or biopolymer---could be used in a rapid scheme that estimate partial charges, thereby avoiding the need for enumeration of geometries, quantum chemical calculations, or fragmentation or linear-scaling schemes.

## The main idea

Alternative methods that utilize only the molecular topology (or molecular graph) exist. In particular, the [VCharge](http://dx.doi.org/10.1021/ci034148o) scheme of Gilson and coworkers presents a particularly clever scheme, which we briefly overview here. A related scheme, [charge equilibration](https://pubs.acs.org/doi/10.1021/j100161a070) by Rappé and Goddard, presents physical reasoning that inspires this model and is also worth reading, though the charge equilibration approach requires a molecular conformation.

The main idea is to determine, for each atom $i$, two strictly positive parameters, $e_i$ and $s_i$, that govern the flow of partial charge among atoms in a manner that preserves the total charge on the system:
* $e_i$ is a parameter related to the _electronegativity_ of the atom, governing how much it attracts charge
* $s_i$ is a parameter related to the _hardness_ of the atom, or how much it resists accumulating or donating charge

These two parameters are determined by a process described in more detail below.

### Determining partial charges from the electronegativity and hardness parameters

Once determined, an optimization problem is solved to identify the partial atomic charges, $q_i$, on each atom $i$. We minimize

$$U({\bf q}) = \sum_{i=1}^N \left[ e_i q_i +  \frac{1}{2}  s_i q_i^2 \right]$$

subject to the constraint that the total charge, $Q$, is preserved:

$$\sum_{i=1}^N q_i = Q$$

It is straigtforward to solve this using [the method of Lagrange multipliers](https://en.wikipedia.org/wiki/Lagrange_multiplier). 
We first introduce the constraint equation into the optimization equation with the Lagrange multiplier $\lambda$:

\begin{eqnarray}
U({\bf q}) &=& \sum_{i=1}^N \left[ e_i q_i +  \frac{1}{2}  s_i q_i^2\right] - \lambda \, \left( \sum_{j=1}^N q_j - Q \right) \\
&=& \sum_{i=1}^N \left[ (e_i - \lambda) q_i +  \frac{1}{2}  s_i q_i^2 \right] + Q
\end{eqnarray}
We find the minimum by computing the partial derivatives $\partial U / \partial q_i$ and equating them to zero at the solution $q_i^*$:
\begin{eqnarray}
0 = (e_i - \lambda) + s_i q_i^* \\
q_i^* = - \frac{(e_i - \lambda)}{s_i}
\end{eqnarray}

We can now solve for $\lambda$ using the constraint equation:

\begin{eqnarray}
\sum_{i=1}^N q^*_i = Q \\
\sum_{i=1}^N \frac{-e_i + \lambda}{s_i} = Q \\
- \sum_{i=1}^N \frac{e_i}{s_i} + \lambda \sum_{i=1}^N \frac{1}{s_i} = Q \\
\lambda = \frac{Q + \sum\limits_{i=1}^N e_i \, s_i^{-1}}{\sum\limits_{j=1}^N s_j^{-1}}
\end{eqnarray}

This yields the solution

\begin{eqnarray}
q_i^* &=& - e_i s_i^{-1} + \lambda s_i^{-1} \\
&=& - e_i s_i^{-1} + s_i^{-1} \frac{Q + \sum\limits_{i=1}^N e_i \, s_i^{-1}}{\sum\limits_{j=1}^N s_j^{-1}}
\end{eqnarray}

It is readily apparent that, once the $e_i$ and $s_i$ parameters are determined, the partial charges $q_i^*$ for even very large molecules can be rapidly computed.

### Determining the electronegativity and hardness parameters

To determine the the electronegativity $e_i$ and harness $h_i$ parameters, the original VCharge scheme used the following equation

$$e_i = e_i^0 + \sum_{j \in N(i)} w(\epsilon_{ij}) \, f(e^o_i - e^o_j) \sum_{j \in N(i)} \sum_{k \in N(j)} w(\epsilon_{ij}, \epsilon_{kl}) \, f(e^o_i - e^o_k)$$

where $N(i)$ denotes the set of neighbors bonded to atom $j$, $\epsilon_{ij}$ is a discrete category (or bond type) for the edge connecting atoms $i$ and $j$, $w(\epsilon)$ is a weight that depends on this bond category, and $f(x)$ is a nonlinear function.

This scheme has a striking resemblance to a graph convolutional neural network, with two hidden layers that aggregate information from parameters of atoms bonded to each atom (and atoms bonded to \emph{those} atoms) via weights and nonlinearities.
Given the advances in graph convolutional and message-passing neural networks since 2003, the question immediately arises: What if we simply use a modern graph convolutional or message-passing network to determine the $e_i$ and $s_i$?

We explore this question here.

## Graph convolutional and message-passing networks

Of particular note are the papers
* [Neural message passing for quantum chemistry](https://arxiv.org/abs/1704.01212), which presents a unifying framework for graph convolutional networks
* [PotentialNet for molecular property prediction](https://pubs.acs.org/doi/full/10.1021/acscentsci.8b00507), a particular kind of graph convolutional network that appears to show [significant recent success in predicting molecular properties](https://arxiv.org/abs/1903.11789)
* [Atomic hardness](https://www.shodor.org/chemviz/ionization/students/hardness.html) is related to an element's likeliness to form a bond (and accept or donate charge)

### Atom features

If we avoid using features that are altered in different resonance forms, we will not need to average charges over resonance forms of the same molecule, which would be highly advantageous when dealing with very large molecules.

The most important properties to encode come directly from the periodic table, as suggested by [Rappé and Goddard](https://pubs.acs.org/doi/10.1021/j100161a070)
* The [electron affinity](https://en.wikipedia.org/wiki/Electron_affinity) is the amount of energy released or spent when an electron is added to an atom; this is related to the physical concept of the electronegativity $e_i$
* The [ionization potential](https://en.wikipedia.org/wiki/Ionization_energy) is the minimum amount of energy required to remove a loosely bound electron from an atom
* The [electronegativity scale](https://en.wikipedia.org/wiki/Electronegativity) is an average of these two

With just these simple features and the atom element encoding, we are likely able to produce a very useful model that meets or exceeds the quality of VCharge.

DeepChem also computes a [number of atom features](https://deepchem.io/docs/_modules/deepchem/feat/graph_features.html#atom_features) that may be of interest:
* one-hot element encoding
* one-hot degree encoding
* one-hot implicit valence encoding
* one-hot hybridization encoding
* binary aromaticity encoding
* one-hot chirality (R/S) encoding plus a boolean for whether this is a chiral center

### Bond features

We could in principle also incorporate features of individual bonds, as suggested in the [neural message passing scheme](https://arxiv.org/abs/1704.01212). Again, if we avoid using features that are altered in different resonance forms of the molecule, our approach will automatically account for chemically equivalent atoms.

DeepChem computes a [number of bond features](https://deepchem.io/docs/_modules/deepchem/feat/graph_features.html#bond_features):
* one-hot encoding of whether the bond is single, double, triple, or aromatic
* boolean of whether the bond is conjugated or not
* boolean of whether the bond is in a ring or not
* one-hot encoding of whether the bond is a stereobond and whether it is E/Z

## Training and test datasets and loss function

Ultimately, we should explore two or three possible target goals:
* Matching partial atomic charges from a low-quality but useful method, such as the AM1 semiempirical method followed by Mullikan population analysis. This can be computed with the [OpenEye Toolkit option OEAM1Charges](https://docs.eyesopen.com/toolkits/python/quacpactk/OEProtonClasses/OEAM1Charges.html).
* Matching high-quality partial atomic charges for the canonical AM1-BCC ELF10 method. These can also be computed with the [OpenEye Toolkit](https://docs.eyesopen.com/toolkits/python/quacpactk/OEProtonClasses/OEAM1BCCELF10Charges.html).
* Matching partial charges for protein sequences parameterized with existing AMBER force field residue templates
* We might, in a future iteration, try fitting electrostatic potentials directly

Loss functions could be simple sum squared deviations from target partial charges.
For polymers like proteins, we may also want to impose a penalty that prevents individual residue units from deviating too much from integral charges of the appropriate formal charge for the ionization state of the residue.

To easily get going, we might try simply using the 3D conformer downloads of fragment subsets of [ZINC15](https://zinc15.docking.org/tranches/home/#), which contain AMSOL partial charges for mol2 files. These provide URL downloads, like http://files.docking.org/3D/AA/AAML/AAAAML.xaa.mol2.gz

I believe OpenEye has a [database of eMolecules with conformations and charges](https://www.eyesopen.com/database-downloads) available that we can likely get a hold of (millions!).

We can try several approaches for splitting into train and test sets.

## Experiments

In [7]:
# Install conda environment and conda-forge
!wget -c https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!bash Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local

# Install dependencies
!conda config --add channels conda-forge --add channels rdkit
!conda create --yes -n conda python=3.6 numpy tensorflow rdkit
!source activate openmm

--2019-03-31 22:13:32--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

PREFIX=/usr/local
reinstalling: python-3.7.1-h0371630_7 ...
Python 3.7.1
reinstalling: ca-certificates-2018.03.07-0 ...
reinstalling: conda-env-2.6.0-1 ...
reinstalling: libgcc-ng-8.2.0-hdf63c60_1 ...
reinstalling: libstdcxx-ng-8.2.0-hdf63c60_1 ...
reinstalling: libffi-3.2.1-hd88cf55_4 ...
reinstalling: ncurses-6.1-he6710b0_1 ...
reinstalling: openssl-1.1.1a-h7b6447c_0 ...
reinstalling: xz-5.2.4-h14c3975_4 ...
reinstalling: yaml-0.1.7-had09818_2 ...
reinstalling: zlib-1.2.11-h7b6447c_3 ...
reinstalling: libedit-3.1.20170329-h6b74fdf_2 ...
reinstalling: readline-7.0-h7b6447c_5

In [0]:
# Add to path
import sys
sys.path.append('/usr/local/envs/conda/lib/python3.6/site-packages')


In [11]:
# Retrieve training and testing datasets
# For now, we just use a small subset of ZINC15 fragments charged with AMSOL
!curl --remote-time -o train.mol2.gz http://files.docking.org/3D/AA/AAMN/AAAAMN.xaa.mol2.gz
!curl --remote-time -o test.mol2.gz http://files.docking.org/3D/AA/AARN/AAAARN.xaa.mol2.gz
!gunzip -f train.mol2.gz test.mol2.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  139k  100  139k    0     0   869k      0 --:--:-- --:--:-- --:--:--  869k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1186k  100 1186k    0     0  2937k      0 --:--:-- --:--:-- --:--:-- 2937k


In [0]:
from rdkit.Chem.rdmolfiles import MolFromMol2File

In [0]:
rdmol = MolFromMol2File('train.mol2', sanitize=False, removeHs=False, cleanupSubstructures=True)

In [14]:
rdmol

In [16]:
from rdkit.Chem.rdmolfiles import MolToSmiles
MolToSmiles(rdmol)

'[H]N([H])C1=NC2=C(N=C(C([H])([H])[H])C(=O)N2[H])C(=O)N1[H]'

In [26]:
for atom in rdmol.GetAtoms():
  print(atom.GetPropNames())

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
